# Análisis reproducible — predicción de `total_alquileres` por hora

Este cuaderno contiene: EDA, preprocesado mínimo, entrenamiento rápido de una línea base, evaluación y ejemplo de llamada a la API.

Notas:
- Ajusta las rutas si tu CSV está en otra ubicación (`dataset_alquiler.csv` en la raíz del repo).
- El cuaderno está pensado para ejecutarse en un entorno con las dependencias del proyecto instaladas. Si alguna librería falta, instala con `pip install -r requirements.txt`.

In [ ]:
# Cell 2: imports básicos
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_theme(style="whitegrid")

In [ ]:
# Cell 3: cargar datos (intenta detectar columna de fecha)
DATA_PATH = Path('..') / 'dataset_alquiler.csv' if Path('..').joinpath('dataset_alquiler.csv').exists() else Path('dataset_alquiler.csv')
print('Leyendo:', DATA_PATH)
df = pd.read_csv(DATA_PATH, low_memory=False)
# detectar columna fecha (si existe)
for col in ['fecha', 'Fecha', 'date', 'timestamp']:
    if col in df.columns:
        try:
            df[col] = pd.to_datetime(df[col])
            df = df.rename(columns={col: 'fecha'})
            break
        except Exception:
            pass

print('Shape:', df.shape)

In [ ]:
# Cell 4: vista rápida
display(df.head())
display(df.describe(include='all'))
display(df.info())

## Preprocesado rápido
Intentamos usar la función `prepare_X_y` de `src/preprocessing.py`. Si no está disponible, aplicamos un preprocesado mínimo: eliminar columnas `u_casuales` y `u_registrados`, crear características horarias e imputar nulos.

In [ ]:
# Cell 5: preprocesado (intento usar util del repo)
try:
    from src.preprocessing import prepare_X_y
    X, y = prepare_X_y(df, drop_cols=['u_casuales','u_registrados'])
    print('Usando prepare_X_y de src.preprocessing')
except Exception as e:
    print('prepare_X_y no disponible o falló:', e)
    df2 = df.copy()
    for c in ['u_casuales','u_registrados']:
        if c in df2.columns:
            df2 = df2.drop(columns=[c])
    if 'fecha' in df2.columns:
        df2['hour'] = df2['fecha'].dt.hour
        df2['dayofweek'] = df2['fecha'].dt.dayofweek
    # objetivo esperado: 'total_alquileres'
    target = 'total_alquileres' if 'total_alquileres' in df2.columns else df2.columns[-1]
    y = df2[target].fillna(0)
    X = df2.select_dtypes(include=[np.number]).drop(columns=[target], errors='ignore').fillna(0)
    print('Preprocesado mínimo completado. X.shape, y.shape:', X.shape, y.shape)

In [ ]:
# Cell 6: entrenamiento rápido (baseline)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model = RandomForestRegressor(n_estimators=50, random_state=42, n_jobs=-1)
model.fit(X_train, y_train)
pred = model.predict(X_test)
rmse = mean_squared_error(y_test, pred, squared=False)
print(f'Baseline RandomForest RMSE: {rmse:.4f}')

In [ ]:
# Cell 7: guardar modelo rápido (opcional)
from joblib import dump
out_dir = Path('models')
out_dir.mkdir(exist_ok=True)
dump(model, out_dir / 'notebook_baseline.joblib')
print('Modelo guardado en models/notebook_baseline.joblib')

## Llamada de ejemplo a la API (desde Python)
El siguiente fragmento muestra cómo llamar al endpoint `/predict` del FastAPI que está en este repo. Ajusta la URL si corres en Docker o en otro host.

In [ ]:
# Cell 8: ejemplo de request a la API
import requests
example_feature = dict(zip(X.columns.tolist(), X.iloc[0].tolist()))
url = 'http://127.0.0.1:8000/predict'
try:
    r = requests.post(url, json={'model':'model_catboost_final','features': example_feature}, timeout=5)
    print('Status:', r.status_code)
    print(r.json())
except Exception as e:
    print('No se pudo conectar a la API (¿está corriendo?):', e)

---
Fin del cuaderno de análisis rápido. Si quieres, puedo:
- Añadir gráficos de SHAP para el modelo final (necesita `shap` y el modelo XGBoost/CatBoost).
- Extender la sección de EDA con series temporales por hora y por día.
- Incluir una celda que descargue y muestre `reports/comparison_table.csv`.